# Analiza podataka iz eksperimenta

In [32]:
import glob
import xlsxwriter
import ipynb.fs.full.functions as functions
import sympy as sp
import numpy as np

x = sp.Symbol('x')
FOLDER = "user_input/*.txt"
HEADERS = ["username","function","pointing_device", "time[s]", "error_rate"]

functions = {
    "e1": functions.easy_fun1,
    "e2": functions.easy_fun2,
    "e3": functions.easy_fun3,
    "e4": functions.easy_fun4,
    
    "m1": functions.medium_fun1,
    "m2": functions.medium_fun2,
    "m3": functions.medium_fun3,
    "m4": functions.medium_fun4,
    
    "h1": functions.hard_fun1,
    "h2": functions.hard_fun2,
    "h3": functions.hard_fun3,
    "h4": functions.hard_fun4,
}

## Obrada podataka iz datoteka

In [33]:
def format_data():
    file_list = glob.glob(FOLDER)
    data = []
    data.append(HEADERS)
    
    for i in file_list:
        points = []
        pointer = ""
        time = 0
        error_rate = 0

        f = open(i, "r")

        username = i.replace("user_input\\", "").replace(".txt", "")[:-3]
        function = i.replace("user_input\\", "").replace(".txt", "")[-2:]

        lines = f.readlines()
        counter = 1
        for l in lines:
            if counter == 1:
                pointer = l[:-1]
            elif counter == 2:
                time = l[:-1]
            else:
                point = l.split()
                point[0] = float(point[0])
                point[1] = float(point[1])
                points.append(point)

            counter += 1
        error_rate = calculate_error(function, points)
        data.append([username, function, pointer, time, error_rate])

    return data

## Izračun greške

In [29]:
def calculate_error(function_label, points):
    function_expr = functions[function_label]
    fun = sp.lambdify(x, function_expr(), "numpy")
    num_of_points = len(points)
    err_i = []
    for p in points:
        r = p[0]
        theta = p[1]
        real_theta = fun(r)
        err_i.append(np.abs(theta - real_theta))
    
    error = 0
    for i in err_i:
        error += i
        
    error = error / num_of_points
    
    return error

## Pohrana podataka u excel

In [11]:
def save_to_xlsx_file(data, name):
    workbook = xlsxwriter.Workbook(name)
    worksheet = workbook.add_worksheet()
    counter = 0
    for i in data:
        worksheet.write_row(counter, 0, i)
        counter += 1
        
    workbook.close()

## Pokretanje analize

In [30]:
data = format_data()
sorted_by_username = sorted(data, key=lambda x : x[0])
save_to_xlsx_file(data, "unsorted_experiment_data.xlsx")